In [16]:
import pandas as pd
import numpy as np 
import folium
folium.__version__ == '0.5.0'

True

In [12]:
data = 'Data/OTC_ACCIDENTS.csv'
df = pd.read_csv(data, sep=';', encoding='latin-1')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19231 entries, 0 to 19230
Data columns (total 35 columns):
ID_ACCIDENT              19231 non-null float64
DATE_                    19231 non-null object
GROUPE_ACCIDENT          19231 non-null object
CAUSE                    19231 non-null object
COMMUNE                  19231 non-null object
CONDITIONS_LUMINEUSES    19231 non-null object
CONDITIONS_METEO         19231 non-null object
CONSEQUENCES             19231 non-null object
COOR_X                   19231 non-null float64
COOR_Y                   19231 non-null float64
ETAT_ROUTE               19231 non-null object
GENRE_ROUTE              19231 non-null object
HEURE                    19230 non-null object
JOUR                     19231 non-null object
LOCALITE                 19231 non-null object
NB_ENFANTS_IMPLIQUES     19231 non-null float64
NB_ENFANTS_ECOLE         19231 non-null float64
NB_BLESSES_LEGERS        19231 non-null float64
NB_BLESSES_GRAVES        19231 non-null

In [15]:
df[['COOR_X','COOR_Y']]

,COOR_X,COOR_Y
0,2500774.0,1117364.0
1,2498974.0,1118100.0
2,2504618.0,1119635.0
3,2502870.0,1118194.0
4,2502440.0,1117246.0
5,2503732.0,1120928.0
6,2499130.0,1116722.0
7,2500768.0,1118614.0
8,2503019.0,1117108.0
9,2498946.0,1119572.0


In [18]:
lausanne_coord = [2500245.0, 1115804.0]
m = folium.Map(location=lausanne_coord)
m